In [14]:

from selenium import webdriver
from selenium.webdriver.common.by import By
import requests
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import pandas as pd



# Web Scraping

I will be getting reviews from goodreads.com. Goodreads is a virtual bookshelf, where you can see the books your friends are reading and vice versa. This is a huge online community of book readers.  

In [16]:
# initialize webdriver
driver = webdriver.Chrome()
url = "https://www.goodreads.com/book/show/62296528/reviews"
driver.get(url)
response = requests.get(url)
time.sleep(5)
show_more_button = driver.find_element(by = By.XPATH, value = '//*[@id="__next"]/div/main/div[1]/div[2]/div[4]/div[4]/div/button')
counter=0
# click the "Show more reviews" button repeatedly until no more reviews are loaded
while counter<40:
    counter = counter+1
    try:
        if show_more_button.is_displayed() and show_more_button.is_enabled():
            driver.execute_script("arguments[0].click();", show_more_button)
            time.sleep(5)
        else:
            break
    except NoSuchElementException:
        break
    except ElementNotInteractableException:
        time.sleep(4)
soup = BeautifulSoup(driver.page_source, 'html.parser')
reviews = soup.find_all('section', 'ReviewText__content')


# find all review blocks
review_blocks = soup.find_all('section', {'class': 'ReviewCard__content'})

# extract the reviews with ratings
reviews_with_ratings = []
for block in review_blocks:
    rating_span = block.find('span', {'class': 'RatingStars RatingStars__small', 'aria-label': lambda value: value and value.startswith('Rating')})
    if rating_span is not None:
        rating = rating_span['aria-label']
        reviews = block.find('section', {'class': 'ReviewText'}).text
        reviews_with_ratings.append({'rating': rating, 'reviews': reviews})

# convert the list of tuples to a pandas DataFrame
df = pd.DataFrame(reviews_with_ratings, columns=['rating', 'reviews'])

In [12]:
df.info

<bound method DataFrame.info of                  rating                                            reviews
0     Rating 1 out of 5  This is the most difficult review I’ve ever wr...
1     Rating 5 out of 5  I really enjoyed this on audio!! Over 15 hours...
2     Rating 3 out of 5  3.5 ⭐⭐⭐ I listened to Prince Harry reading thi...
3     Rating 3 out of 5  My curiosity got the better of me and I read P...
4     Rating 3 out of 5  So, I have always “liked” Harry. This memoir m...
...                 ...                                                ...
1110  Rating 2 out of 5  Rating: 2.5 I’ll admit it. The hype got the be...
1111  Rating 4 out of 5  If you go into “Spare” baring negative preconc...
1112  Rating 2 out of 5  okay so i don't really give a shit about the r...
1113  Rating 4 out of 5  3,5 stjärnor. Är man inte intresserad av kunga...
1114  Rating 2 out of 5  The first 60% of this book was a slog to get t...

[1115 rows x 2 columns]>

In [13]:
df.to_csv('web_content.csv',index=False)